In [0]:
#import required libraries
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from geopy.geocoders import Nominatim
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [0]:
#load the dataset 'yellow_tripdata_2019-06.csv'
from google.colab import files
uploaded = files.upload()

Saving yellow_tripdata_2019-06 (1).csv to yellow_tripdata_2019-06 (1).csv


In [0]:
#read the csv file to a pandas dataframe
sample_df = pd.read_csv("yellow_tripdata_2019-06 (1).csv")
sample_df.shape

(6941024, 18)

In [0]:
sample_df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [0]:
df = sample_df.drop(['VendorID','RatecodeID','passenger_count','payment_type','fare_amount','total_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','store_and_fwd_flag'], axis = 1)
df.dtypes

tpep_pickup_datetime      object
tpep_dropoff_datetime     object
trip_distance            float64
PULocationID               int64
DOLocationID               int64
dtype: object

In [0]:
#save pre-processed data to new csv file for easy retrieval
df.to_csv('yellow_tripdata_2019-06.csv')
!cp yellow_tripdata_2019-06.csv drive/My\ Drive/

In [0]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math

from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
# Download the location Data
urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip", "taxi_zones.zip")
with zipfile.ZipFile("taxi_zones.zip","r") as zip_ref:
    zip_ref.extractall("./shape")

In [0]:
pip install pyshp

In [0]:
import shapefile

In [0]:
def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

In [0]:
sf = shapefile.Reader("shape/taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]

df_loc = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")
df_loc.head(10)

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,longitude,latitude
0,1,0.116357,0.000782,Newark Airport,1,EWR,9.366817e+05,190522.130278
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,1.033536e+06,161853.982300
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,1.027136e+06,254730.010849
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,9.904240e+05,203100.040432
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,9.321332e+05,139954.541936
5,6,0.150491,0.000606,Arrochar/Fort Wadsworth,6,Staten Island,9.640990e+05,157533.672063
6,7,0.107417,0.000390,Astoria,7,Queens,1.006325e+06,216569.411021
7,8,0.027591,0.000027,Astoria Park,8,Queens,1.005542e+06,222960.582314
8,9,0.099784,0.000338,Auburndale,9,Queens,1.043404e+06,214170.702621
9,10,0.099839,0.000436,Baisley Park,10,Queens,1.042776e+06,186452.538244


In [0]:
df_loc = df_loc.drop(['Shape_Area','Shape_Leng','borough','OBJECTID'], axis = 1)

In [0]:
df_loc.rename(columns = {'LocationID':'PULocationID'}, inplace = True)
df_loc.rename(columns = {'longitude':'PUlongitude'}, inplace = True)
df_loc.rename(columns = {'latitude':'PUlatitude'}, inplace = True)

In [0]:
df = pd.merge(df, df_loc, on='PULocationID')

In [0]:
df_loc.rename(columns = {'PULocationID':'DOLocationID'}, inplace = True)
df_loc.rename(columns = {'PUlongitude':'DOlongitude'}, inplace = True)
df_loc.rename(columns = {'PUlatitude':'DOlatitude'}, inplace = True)

In [0]:
df = pd.merge(df, df_loc, on='DOLocationID')

In [0]:
df.head(10)

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,zone_x,PUlongitude,PUlatitude,zone_y,DOlongitude,DOlatitude
0,2019-06-01 00:55:13,2019-06-01 00:56:17,0.00,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978
1,2019-06-01 00:59:02,2019-06-01 00:59:12,0.80,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978
2,2019-06-01 00:08:05,2019-06-01 00:11:53,0.54,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978
3,2019-06-01 00:51:18,2019-06-01 00:53:14,0.40,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978
4,2019-06-01 01:20:55,2019-06-01 01:21:09,0.00,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978
5,2019-06-01 01:21:51,2019-06-01 01:21:57,0.00,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978
6,2019-06-01 02:55:09,2019-06-01 02:56:32,0.00,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978
7,2019-06-01 02:08:32,2019-06-01 02:08:45,0.00,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978
8,2019-06-01 03:10:54,2019-06-01 03:28:08,0.20,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978
9,2019-06-01 04:36:03,2019-06-01 04:36:17,0.00,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978


In [0]:
#Convert datetime strings into datetime
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], format='%Y-%m-%d %H:%M:%S')
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"], format='%Y-%m-%d %H:%M:%S')

In [0]:
#Create variables, like month, date, etc.
df["pickup_month"] = df["tpep_pickup_datetime"].dt.month
df["pickup_day"] = df["tpep_pickup_datetime"].dt.day
df["pickup_weekday"] = df["tpep_pickup_datetime"].dt.weekday 
df["pickup_hour"] = df["tpep_pickup_datetime"].dt.hour
df["pickup_minute"] = df["tpep_pickup_datetime"].dt.minute

In [0]:
#Get latitude and longitude differences 
df["latitude_difference"] = df["DOlatitude"] - df["PUlatitude"]
df["longitude_difference"] = df["DOlongitude"] - df["PUlongitude"]

In [0]:
#Convert trip distance from longitude and latitude differences to Manhattan distance.
df["trip_distance"] = 0.621371 * 6371 * (abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(df["latitude_difference"]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(df["latitude_difference"]) * np.pi / 180) / 2)))))) + \
                                     abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(df["longitude_difference"]) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(df["longitude_difference"]) * np.pi / 180) / 2)))))))

In [0]:
df["trip_duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]

In [0]:
df["trip_duration"] = df["trip_duration"].dt.seconds

In [0]:
df["trip_duration"] = df["trip_duration"].apply(lambda x: round(x/60))

In [0]:
df.head(10)

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,zone_x,PUlongitude,PUlatitude,zone_y,DOlongitude,DOlatitude,pickup_month,pickup_day,pickup_weekday,pickup_hour,pickup_minute,latitude_difference,longitude_difference,trip_duration
0,2019-06-01 00:55:13,2019-06-01 00:56:17,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,0,55,0.0,0.0,1
1,2019-06-01 00:59:02,2019-06-01 00:59:12,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,0,59,0.0,0.0,0
2,2019-06-01 00:08:05,2019-06-01 00:11:53,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,0,8,0.0,0.0,4
3,2019-06-01 00:51:18,2019-06-01 00:53:14,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,0,51,0.0,0.0,2
4,2019-06-01 01:20:55,2019-06-01 01:21:09,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,1,20,0.0,0.0,0
5,2019-06-01 01:21:51,2019-06-01 01:21:57,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,1,21,0.0,0.0,0
6,2019-06-01 02:55:09,2019-06-01 02:56:32,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,2,55,0.0,0.0,1
7,2019-06-01 02:08:32,2019-06-01 02:08:45,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,2,8,0.0,0.0,0
8,2019-06-01 03:10:54,2019-06-01 03:28:08,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,3,10,0.0,0.0,17
9,2019-06-01 04:36:03,2019-06-01 04:36:17,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,4,36,0.0,0.0,0


In [0]:
#save pre-processed data to new csv file for easy retrieval
df.to_csv('preprocessed_data.csv')
!cp preprocessed_data.csv drive/My\ Drive/

In [0]:
dfnew = pd.read_csv("/content/drive/My Drive/preprocessed_data.csv", index_col=[0])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
dfnew.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,zone_x,PUlongitude,PUlatitude,zone_y,DOlongitude,DOlatitude,pickup_month,pickup_day,pickup_weekday,pickup_hour,pickup_minute,latitude_difference,longitude_difference,trip_duration
0,2019-06-01 00:55:13,2019-06-01 00:56:17,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,0,55,0.0,0.0,1
1,2019-06-01 00:59:02,2019-06-01 00:59:12,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,0,59,0.0,0.0,0
2,2019-06-01 00:08:05,2019-06-01 00:11:53,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,0,8,0.0,0.0,4
3,2019-06-01 00:51:18,2019-06-01 00:53:14,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,0,51,0.0,0.0,2
4,2019-06-01 01:20:55,2019-06-01 01:21:09,0.0,145,145,Long Island City/Hunters Point,998207.57297,211280.687978,Long Island City/Hunters Point,998207.57297,211280.687978,6,1,5,1,20,0.0,0.0,0


In [0]:
# Define input X and output Y for the XGBoost model
# Trip Duration is the output and all the remaining attributes 
# of preprocessed dataset are the input features.
y = dfnew["trip_duration"]
df_X = dfnew.drop(["trip_duration","tpep_pickup_datetime", "tpep_dropoff_datetime","PULocationID", "DOLocationID","zone_x","zone_y","tpep_pickup_datetime","tpep_dropoff_datetime"], axis=1)

In [0]:
X = df_X

In [0]:
#Split the data into training, test, and valdiation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

In [0]:
#Define evaluation metric
def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean() ** 0.5

In [0]:
#XGBoost parameters 
params = {
    'booster':            'gbtree',
    'objective':          'reg:linear',
    'learning_rate':      0.05,
    'max_depth':          10,
    'subsample':          0.8,
    'colsample_bytree':   0.7,
    'colsample_bylevel':  0.7,
    'silent':             1,
    'feval':              'rmsle'
}

In [0]:
nrounds = 500

In [0]:
#Define train and validation sets
dtrain = xgb.DMatrix(X_train, np.log(y_train+1))
dval = xgb.DMatrix(X_val, np.log(y_val+1))

#log the error
watchlist = [(dval, 'eval'), (dtrain, 'train')]

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [0]:
#Train model
gbm = xgb.train(params,
                dtrain,
                num_boost_round = nrounds,
                evals = watchlist,
                verbose_eval = True
                )

[0]	eval-rmse:2.0726	train-rmse:2.07264
[1]	eval-rmse:1.97493	train-rmse:1.97497
[2]	eval-rmse:1.88263	train-rmse:1.88267
[3]	eval-rmse:1.79545	train-rmse:1.79548
[4]	eval-rmse:1.71244	train-rmse:1.71247
[5]	eval-rmse:1.63366	train-rmse:1.63368
[6]	eval-rmse:1.56	train-rmse:1.56002
[7]	eval-rmse:1.49002	train-rmse:1.49003
[8]	eval-rmse:1.42394	train-rmse:1.42395
[9]	eval-rmse:1.3614	train-rmse:1.36141
[10]	eval-rmse:1.30133	train-rmse:1.30132
[11]	eval-rmse:1.24563	train-rmse:1.24562
[12]	eval-rmse:1.19314	train-rmse:1.19311
[13]	eval-rmse:1.14265	train-rmse:1.14261
[14]	eval-rmse:1.09569	train-rmse:1.09564
[15]	eval-rmse:1.0511	train-rmse:1.05102
[16]	eval-rmse:1.0088	train-rmse:1.00869
[17]	eval-rmse:0.96972	train-rmse:0.969599
[18]	eval-rmse:0.932786	train-rmse:0.932642
[19]	eval-rmse:0.898565	train-rmse:0.898398
[20]	eval-rmse:0.865015	train-rmse:0.864814
[21]	eval-rmse:0.834203	train-rmse:0.833977
[22]	eval-rmse:0.805468	train-rmse:0.805212
[23]	eval-rmse:0.778578	train-rmse:0.778

In [0]:
#Test predictions
pred = np.exp(gbm.predict(xgb.DMatrix(X_test))) - 1

In [0]:
from sklearn.metrics import mean_absolute_error
# Compute MAE
mae_baseline = mean_absolute_error(y_test, pred)
print("MAE for XGBoost is {:.2f}".format(mae_baseline))

MAE for XGBoost is 6.95


In [0]:
#feature importance
feature_scores = gbm.get_fscore()
feature_scores

{'DOlatitude': 31287,
 'DOlongitude': 32641,
 'PUlatitude': 33078,
 'PUlongitude': 31646,
 'latitude_difference': 34036,
 'longitude_difference': 34166,
 'pickup_day': 43164,
 'pickup_hour': 42323,
 'pickup_minute': 42787,
 'pickup_month': 749,
 'pickup_weekday': 26516,
 'trip_distance': 35151}

In [0]:
filename = "xgb_model.sav"
pickle.dump(gbm, open(filename, 'wb'))

In [0]:
# Toy example
# Create a dictionary of 11 destination points for which 
# an optimal travel route is to be computed.

test_locations11 = {'L1': (1.033536e+06,	161853.982300),
                  'L2': (1.027136e+06,	254730.010849),
                  'L3': (9.904240e+05,	203100.040432),
                  'L4': (9.798897e+05,	195215.447490),
                  'L5': (9.640990e+05,	157533.672063),
                  'L6': (9.366817e+05,	190522.130278),
                  'L7': (1.006325e+06,	216569.411021),
                  'L8': (1.005542e+06,	222960.582314),
                  'L9': (1.043404e+06,	214170.702621),
                  'L10': (1.042776e+06,	186452.538244),
                  'L11': (9.821645e+05,	159335.254890)
             }

In [0]:
test_locations25 = {'L1': (979889.680921644,	166469.262621104),
                  'L2': (1043719.66511429,	225304.689078733),
                  'L3': (1046653.02704402,	216690.187322497),
                  'L4': (997925.493874393,	191314.483538232),
                  'L5': (1014322.51449931,	255779.454359598),
                  'L6': (1059467.43583507,	207808.57245142),
                  'L7': (1015730.95635821,	251796.605428755),
                  'L8': (988177.344968885,	158599.73524519),
                  'L9': (985618.376440212,	161399.825120113),
                  'L10': (939687.959780715,	160650.812143862),
                  'L11': (993795.402611047,	231438.438452616),
                  'L12': (988001.907131552, 189102.798642702),
                  'L13': (986990.642434284, 168293.055545292),
                  'L14': (1011562.84827313, 142931.067955672),
                  'L15': (1037383.3421418, 198281.296653412),
                  'L16': (995188.676321141, 151504.591864936),
                  'L17': (1032900.68284718, 159709.402377381),
                  'L18': (1018522.63383613, 252630.069383665),
                  'L19': (1021909.61174634, 254224.004557296),
                  'L20': (985353.255190193, 192941.713096439),
                  'L21': (991925.084378376, 195220.455128856),
                  'L22': (1008500.42819876, 181131.110185876),
                  'L23': (1007105.37338669, 193942.627648338),
                  'L24': (1006317.79843635, 192966.719369776),
                  'L25': (1057249.34161752, 191999.301776528)}

In [0]:
# Sample date as input to the XGBoost model in addition to above test_locations
from copy import copy
import datetime
import pickle
from geopy.geocoders import Nominatim

date_list = [24, 11, 2019] #Nov 24, 2019

year = int(date_list[2])
month = int(date_list[1])
day = int(date_list[0])

my_date = datetime.date(year, month, day)

In [0]:
# invoke the trained XGBoost model to predict trip duration between each pair of points
# in sample test_location
def travel_time_between_points(PUlongitude, PUlatitude, DOlongitude, DOlatitude, hour, date,
                               pickup_minute = 0):
    """
    Given two points, this calculates travel between them based on a XGBoost predictive model
    """
    
    model_data = {'trip_distance' : 0.621371 * 6371 * (abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(DOlatitude - PUlatitude) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(DOlatitude - PUlatitude) * np.pi / 180) / 2)))))) + \
                                     abs(2 * np.arctan2(np.sqrt(np.square(np.sin((abs(DOlongitude - PUlongitude) * np.pi / 180) / 2))), 
                                  np.sqrt(1-(np.square(np.sin((abs(DOlongitude - PUlongitude) * np.pi / 180) / 2))))))),
                  'PUlongitude' : PUlongitude, 
                  'PUlatitude' : PUlatitude, 
                  'DOlongitude' : DOlongitude, 
                  'DOlatitude' : DOlatitude, 
                  'pickup_month' : my_date.month,
                  'pickup_day' : my_date.day,
                  'pickup_weekday' : my_date.weekday(),
                  'pickup_hour': hour,
                  'pickup_minute' : pickup_minute,
                  'latitude_difference' : DOlatitude - PUlatitude,
                  'longitude_difference' : DOlongitude - PUlongitude,
                 }
    #print(model_data)
    df = pd.DataFrame([model_data], columns=model_data.keys())
    
    pred = np.exp(gbm.predict(xgb.DMatrix(df))) - 1
    #print(pred)
    return pred[0]

In [0]:
# Create a lookup table of travel time between each pair of points
# to feed as input to the genetic algorithm
data = []
for i in test_locations11:
  PU = []
  for value in test_locations11[i]:
    PU.append(value)
  for j in test_locations11:
    DO = []
    for valuee in test_locations11[j]:
      DO.append(valuee) 
      #print(PU[0], PU[1], DO[0], DO[1])
    trip_duration = travel_time_between_points(PU[0], PU[1], DO[0], DO[1], 11, my_date)
    data.append((PU[0], PU[1], DO[0], DO[1], trip_duration))
  

df_final = pd.DataFrame(data, columns = ['PUlongitude', 'PUlatitude','DOlongitude','DOlatitude','Trip_Duration']) 
df_final
      



    

,PUlongitude,PUlatitude,DOlongitude,DOlatitude,Trip_Duration
0,1033536.0,161853.98230,1033536.0,161853.982300,4.297942
1,1033536.0,161853.98230,1027136.0,254730.010849,54.873241
2,1033536.0,161853.98230,990424.0,203100.040432,69.921715
3,1033536.0,161853.98230,979889.7,195215.447490,93.690651
4,1033536.0,161853.98230,964099.0,157533.672063,60.947159
...,...,...,...,...,...
116,982164.5,159335.25489,1006325.0,216569.411021,95.749710
117,982164.5,159335.25489,1005542.0,222960.582314,76.221634
118,982164.5,159335.25489,1043404.0,214170.702621,131.365021
119,982164.5,159335.25489,1042776.0,186452.538244,113.204300


In [0]:
df_final.to_csv("xgboost_table.csv")

In [0]:
# Create a lookup table of travel time between each pair of points
# to feed as input to the genetic algorithm
data = []
for i in test_locations25:
  PU = []
  for value in test_locations25[i]:
    PU.append(value)
  for j in test_locations25:
    DO = []
    for valuee in test_locations25[j]:
      DO.append(valuee) 
      #print(PU[0], PU[1], DO[0], DO[1])
    trip_duration = travel_time_between_points(PU[0], PU[1], DO[0], DO[1], 11, my_date)
    data.append((PU[0], PU[1], DO[0], DO[1], trip_duration))
  

df_final = pd.DataFrame(data, columns = ['PUlongitude', 'PUlatitude','DOlongitude','DOlatitude','Trip_Duration']) 
df_final

,PUlongitude,PUlatitude,DOlongitude,DOlatitude,Trip_Duration
0,9.798897e+05,166469.262621,9.798897e+05,166469.262621,2.740764
1,9.798897e+05,166469.262621,1.043720e+06,225304.689079,143.004745
2,9.798897e+05,166469.262621,1.046653e+06,216690.187322,125.105591
3,9.798897e+05,166469.262621,9.979255e+05,191314.483538,71.487793
4,9.798897e+05,166469.262621,1.014323e+06,255779.454360,105.265152
...,...,...,...,...,...
620,1.057249e+06,191999.301777,9.919251e+05,195220.455129,72.200020
621,1.057249e+06,191999.301777,1.008500e+06,181131.110186,84.550041
622,1.057249e+06,191999.301777,1.007105e+06,193942.627648,58.352272
623,1.057249e+06,191999.301777,1.006318e+06,192966.719370,54.294865
